In [1]:
import os
import platform
import numpy as np
import pandas as pd

In [2]:
print(f"Conda env      = {os.environ['CONDA_DEFAULT_ENV']}")
print(f"Python version = {platform.python_version()}")
print(f"Pandas version = {pd.__version__}")

Conda env      = pandas
Python version = 3.9.16
Pandas version = 2.0.1


In [3]:
def profile_header():
    print(f"{'column':21} {'type':>15} {'unique':>8} {'mem_mb':>8}")
    
def profile_column(col):
        name = col.name
        typ = str(col.dtype)
        n_unique = len(col.unique())
        mem_mb = col.memory_usage(deep=True) // (1024**2)               
        print(f"{name:21} {typ:>15} {n_unique:8} {mem_mb:8}")    

def df_mb(df):
    sum = 0
    for col in df.columns:
        sum += df[col].memory_usage(deep=True)
    return sum // (1024**2)
        
        
def profile(df):
    print(f"DF nrows  = {len(df):,}")      
    print(f"DF memory = {df_mb(df):,} MB")  
    profile_header()    
    for col in df.columns:
        profile_column(df[col])

In [4]:
%%time

# Read uncompressed csv data w/o parsing dates, note the size
df0 = pd.read_csv("yellow_tripdata_2020-01-1M.csv")
profile(df0)

DF nrows  = 1,000,000
DF memory = 320 MB
column                           type   unique   mem_mb
VendorID                      float64        3        7
tpep_pickup_datetime           object   386921       75
tpep_dropoff_datetime          object   388189       75
passenger_count               float64       11        7
trip_distance                 float64     3412        7
RatecodeID                    float64        8        7
store_and_fwd_flag             object        3       55
PULocationID                    int64      251        7
DOLocationID                    int64      260        7
payment_type                  float64        5        7
fare_amount                   float64     1626        7
extra                         float64       23        7
mta_tax                       float64        5        7
tip_amount                    float64     2202        7
tolls_amount                  float64      447        7
improvement_surcharge         float64        3        7
total_a

In [5]:
%%time

# Read uncompressed csv data w/o parsing dates, note the size
# Also, used nullable types
df0_1 = pd.read_csv(
    "yellow_tripdata_2020-01-1M.csv",
    dtype_backend="numpy_nullable"
)

profile(df0_1)

DF nrows  = 1,000,000
DF memory = 334 MB
column                           type   unique   mem_mb
VendorID                        Int64        3        8
tpep_pickup_datetime           string   386921       75
tpep_dropoff_datetime          string   388189       75
passenger_count                 Int64       11        8
trip_distance                 Float64     3412        8
RatecodeID                      Int64        8        8
store_and_fwd_flag             string        3       55
PULocationID                    Int64      251        8
DOLocationID                    Int64      260        8
payment_type                    Int64        5        8
fare_amount                   Float64     1626        8
extra                         Float64       23        8
mta_tax                       Float64        5        8
tip_amount                    Float64     2202        8
tolls_amount                  Float64      447        8
improvement_surcharge         Float64        3        8
total_a

In [4]:
%%time

# parse dates so they take up less space (75MB->7MB)- LONG TIME TO CONVERT (2m28s vs 1.8s)
df1 = pd.read_csv(
    "yellow_tripdata_2020-01-1M.csv", 
     parse_dates=[
         'tpep_pickup_datetime', 
         'tpep_dropoff_datetime'
     ], 
     infer_datetime_format=True
)
profile(df1)

DF nrows  = 1,000,000
DF memory = 184 MB
column                           type   unique   mem_mb
VendorID                      float64        3        7
tpep_pickup_datetime   datetime64[ns]   386921        7
tpep_dropoff_datetime  datetime64[ns]   388189        7
passenger_count               float64       11        7
trip_distance                 float64     3412        7
RatecodeID                    float64        8        7
store_and_fwd_flag             object        3       55
PULocationID                    int64      251        7
DOLocationID                    int64      260        7
payment_type                  float64        5        7
fare_amount                   float64     1626        7
extra                         float64       23        7
mta_tax                       float64        5        7
tip_amount                    float64     2202        7
tolls_amount                  float64      447        7
improvement_surcharge         float64        3        7
total_a

In [5]:
%%time

# Read compressed data - not converting dates (3s compressed vs 1.8s uncompressed)
df2 = pd.read_csv("yellow_tripdata_2020-01-1M.csv.xz")
profile(df2)

DF nrows  = 1,000,000
DF memory = 320 MB
column                           type   unique   mem_mb
VendorID                      float64        3        7
tpep_pickup_datetime           object   386921       75
tpep_dropoff_datetime          object   388189       75
passenger_count               float64       11        7
trip_distance                 float64     3412        7
RatecodeID                    float64        8        7
store_and_fwd_flag             object        3       55
PULocationID                    int64      251        7
DOLocationID                    int64      260        7
payment_type                  float64        5        7
fare_amount                   float64     1626        7
extra                         float64       23        7
mta_tax                       float64        5        7
tip_amount                    float64     2202        7
tolls_amount                  float64      447        7
improvement_surcharge         float64        3        7
total_a

In [6]:
%%time 

# Reduce size by converting from object to datetime (also read_csv optiona)
# df3 = df2.copy(deep=True)
# df3["tpep_pickup_datetime"]  = pd.to_datetime(df3["tpep_pickup_datetime"])
# df3["tpep_dropoff_datetime"] = pd.to_datetime(df3["tpep_dropoff_datetime"])
# profile(df3)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.29 µs


In [7]:
# Reduce size by converting low cardinality floats (integers w/ Nan's) to nullable integers
import numpy as np

df4 = df1.copy(deep=True)
profile_header()
profile_column(df4["payment_type"])
print(df4["payment_type"].unique())

print(f"df before = {df_mb(df4):,} MB")
df4["payment_type"] = df4["payment_type"].astype('Int8')
profile_column(df4["payment_type"])
print(f"df after = {df_mb(df4):,} MB")

column                           type   unique   mem_mb
payment_type                  float64        5        7
[ 1.  2.  4.  3. nan]
df before = 184 MB
payment_type                     Int8        5        1
df after = 179 MB


In [11]:
%%time

# Read parquet - 6.4MRows in 4s, picks up datetime (could specify entire schema)
df_pq = pd.read_parquet(
    "yellow_tripdata_2020-01.parquet", 
    dtype_backend='numpy_nullable',
)
profile(df_pq)

DF nrows  = 6,405,008
DF memory = 1,422 MB
column                           type   unique   mem_mb
VendorID                        Int64        3       54
tpep_pickup_datetime   datetime64[ns]  2134342       48
tpep_dropoff_datetime  datetime64[ns]  2137286       48
passenger_count               Float64       11       54
trip_distance                 Float64     5606       54
RatecodeID                    Float64        8       54
store_and_fwd_flag             string        3      353
PULocationID                    Int64      261       54
DOLocationID                    Int64      262       54
payment_type                    Int64        6       54
fare_amount                   Float64     5283       54
extra                         Float64       47       54
mta_tax                       Float64       11       54
tip_amount                    Float64     3626       54
tolls_amount                  Float64     1035       54
improvement_surcharge         Float64        3       54
total

In [15]:
%%time
print()
df_pq["payment_type"] = df_pq["payment_type"].astype('category')
df_pq["store_and_fwd_flag"] = df_pq["store_and_fwd_flag"].astype('category')
df_pq["PULocationID"] = df_pq["PULocationID"].astype('category')
df_pq["DOLocationID"] = df_pq["DOLocationID"].astype('category')
profile(df_pq)


DF nrows  = 6,405,008
DF memory = 940 MB
column                           type   unique   mem_mb
VendorID                        Int64        3       54
tpep_pickup_datetime   datetime64[ns]  2134342       48
tpep_dropoff_datetime  datetime64[ns]  2137286       48
passenger_count               Float64       11       54
trip_distance                 Float64     5606       54
RatecodeID                    Float64        8       54
store_and_fwd_flag           category        3        6
PULocationID                 category      261       12
DOLocationID                 category      262       12
payment_type                 category        6        6
fare_amount                   Float64     5283       54
extra                         Float64       47       54
mta_tax                       Float64       11       54
tip_amount                    Float64     3626       54
tolls_amount                  Float64     1035       54
improvement_surcharge         Float64        3       54
total_

In [86]:
s = df_pq["airport_fee"]
# print(s)
# print(len(s))

print(s[0])
print(type(s[0]))
s == None

def f(x):
    print(f"x = x, {type(x)}")
    return x == None

# s2 = s.loc[f]
# print(len(s2))

# l = list(s)
# print(l[0])
# print(type(l[0]))

# s[:4].apply(f, axis=1)

df_pq.query("VendorID > 4 order by VendorID")

None
<class 'NoneType'>


SyntaxError: invalid syntax (<unknown>, line 1)

In [26]:
list(df_pq["airport_fee"].filter(items=[None]))

[]

In [33]:
data = pd.Series([4, 7, 7, 12, 19, 23, 25, 30])
data.loc[lambda x : x == 7]

1    7
2    7
dtype: int64